In [1]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [2]:
import pandas as pd
import requests
from io import BytesIO

In [3]:
from data_management.models import *

In [4]:
CroppedImage.objects.select_related('origin_source')

<QuerySet [<CroppedImage: CroppedImage object (178)>, <CroppedImage: CroppedImage object (179)>, <CroppedImage: CroppedImage object (180)>, <CroppedImage: CroppedImage object (181)>, <CroppedImage: CroppedImage object (182)>, <CroppedImage: CroppedImage object (183)>, <CroppedImage: CroppedImage object (184)>, <CroppedImage: CroppedImage object (185)>, <CroppedImage: CroppedImage object (186)>, <CroppedImage: CroppedImage object (187)>, <CroppedImage: CroppedImage object (188)>, <CroppedImage: CroppedImage object (189)>, <CroppedImage: CroppedImage object (190)>, <CroppedImage: CroppedImage object (191)>, <CroppedImage: CroppedImage object (192)>, <CroppedImage: CroppedImage object (193)>, <CroppedImage: CroppedImage object (194)>, <CroppedImage: CroppedImage object (195)>, <CroppedImage: CroppedImage object (196)>, <CroppedImage: CroppedImage object (197)>, '...(remaining elements truncated)...']>

In [8]:
class DataExtractSave():
    def __init__(self):
        self.left = []
        self.top = []
        self.right = []
        self.bottom = []
        self.origin_url = []
        self.width = []
        self.height = []
    
    def get_image_size(self, resp):
        from PIL import Image
        byteImgIO = BytesIO()
        byteImg = Image.open(BytesIO(resp.content))
        byteImg.save(byteImgIO, "JPEG")
        byteImgIO.seek(0)
        byteImg = byteImgIO.read()
        dataBytesIO = BytesIO(byteImg)
        image = Image.open(dataBytesIO)
        image = image.convert('RGB')
        width, height = image.size
        return width, height
    
    def get_data_and_append(self, obj, url):
        self.origin_url.append(url)
        resp = requests.get(url)
        w, h = self.get_image_size(resp)
        self.width.append(w)
        self.height.append(h)
        self.left.append(obj.left*w)
        self.top.append(obj.top*h)
        self.right.append(obj.right*w)
        self.bottom.append(obj.bottom*h)
    
    def save_to_csv(self,  filename):
        dic = {}
        dic['left'] = self.left
        dic['top'] = self.top
        dic['right'] = self.right
        dic['bottom'] = self.bottom
        dic['origin_url'] = self.origin_url
        dic['width'] = self.width
        dic['height'] = self.height
    
        test_data_pd=pd.DataFrame.from_dict(dic)
        print(filename)
        test_data_pd.to_csv(filename, mode='w', index=False)
        print('downloaded')
        
    def save_data_from_cropped(self, filename, n):
        print(n)
        
        if not n:
            queryset = CroppedImage.objects.filter(origin_source__image_url__isnull=False)
        else:
            N = int(n)
            queryset = CroppedImage.objects.filter(origin_source__image_url__isnull=False)[:N]
        i = 0
        for c in queryset:
            i += 1
            print(i)
            print(c)
            if c.origin_source.image:
                url = c.origin_source.image.url
                self.get_data_and_append(c, url)
        self.save_to_csv(filename)
        print('DONE!')
        
    def save_data_from_s3_temp(self, filename, n):       
        if not n:
            queryset = CroppedImage.objects.filter(origin_source__image="")
        else:
            N = int(n)
            queryset = CroppedImage.objects.filter(origin_source__image="")[:N]
            print(queryset)
        i = 0
        for c in queryset:
            i +=1
            print(i)
            print(c)
            if c.origin_source.s3_image_url:
                url = c.origin_source.s3_image_url
                self.get_data_and_append(c, url)
        self.save_to_csv(filename)
        print('DONE!')

In [9]:
# save_data_from_cropped : 크로링 박싱후 저장한 데이터 csv로 저장
# save_data_from_cropped('저장할 파일이름', 쿼리셋 개수) --> 쿼리셋 개수는 비워두면 전체에 대해 다운로드
# Ex: save_data_from_cropped("crop_download_test.csv","")

In [11]:
downloadmanager1= DataExtractSave()

In [12]:
downloadmanager1.save_data_from_cropped("crop_download_test.csv","")


1
CroppedImage object (178)
2
CroppedImage object (179)
3
CroppedImage object (180)
4
CroppedImage object (181)
5
CroppedImage object (182)
6
CroppedImage object (183)
7
CroppedImage object (184)
8
CroppedImage object (185)
9
CroppedImage object (186)
10
CroppedImage object (187)
11
CroppedImage object (188)
12
CroppedImage object (189)
13
CroppedImage object (190)
14
CroppedImage object (191)
15
CroppedImage object (192)
16
CroppedImage object (193)
17
CroppedImage object (194)
18
CroppedImage object (195)
19
CroppedImage object (196)
20
CroppedImage object (197)
21
CroppedImage object (198)
22
CroppedImage object (199)
23
CroppedImage object (200)
24
CroppedImage object (201)
25
CroppedImage object (202)
26
CroppedImage object (203)
27
CroppedImage object (204)
28
CroppedImage object (205)
29
CroppedImage object (206)
30
CroppedImage object (207)
31
CroppedImage object (208)
32
CroppedImage object (209)
33
CroppedImage object (210)
34
CroppedImage object (211)
35
CroppedImage object

279
CroppedImage object (460)
280
CroppedImage object (461)
281
CroppedImage object (462)
282
CroppedImage object (463)
283
CroppedImage object (464)
284
CroppedImage object (465)
285
CroppedImage object (466)
286
CroppedImage object (467)
287
CroppedImage object (468)
288
CroppedImage object (469)
289
CroppedImage object (470)
290
CroppedImage object (471)
291
CroppedImage object (472)
292
CroppedImage object (473)
293
CroppedImage object (474)
294
CroppedImage object (475)
295
CroppedImage object (476)
296
CroppedImage object (477)
297
CroppedImage object (478)
298
CroppedImage object (479)
299
CroppedImage object (480)
300
CroppedImage object (481)
301
CroppedImage object (482)
302
CroppedImage object (483)
303
CroppedImage object (484)
304
CroppedImage object (485)
305
CroppedImage object (486)
306
CroppedImage object (487)
307
CroppedImage object (488)
308
CroppedImage object (489)
309
CroppedImage object (490)
310
CroppedImage object (491)
311
CroppedImage object (492)
312
Croppe

553
CroppedImage object (737)
554
CroppedImage object (738)
555
CroppedImage object (739)
556
CroppedImage object (740)
557
CroppedImage object (741)
558
CroppedImage object (742)
559
CroppedImage object (743)
560
CroppedImage object (744)
561
CroppedImage object (745)
562
CroppedImage object (746)
563
CroppedImage object (748)
564
CroppedImage object (749)
565
CroppedImage object (750)
566
CroppedImage object (751)
567
CroppedImage object (752)
568
CroppedImage object (753)
569
CroppedImage object (754)
570
CroppedImage object (755)
571
CroppedImage object (756)
572
CroppedImage object (757)
573
CroppedImage object (758)
574
CroppedImage object (759)
575
CroppedImage object (760)
576
CroppedImage object (761)
577
CroppedImage object (762)
578
CroppedImage object (763)
579
CroppedImage object (764)
580
CroppedImage object (766)
581
CroppedImage object (767)
582
CroppedImage object (768)
583
CroppedImage object (769)
584
CroppedImage object (770)
585
CroppedImage object (771)
586
Croppe

827
CroppedImage object (1019)
828
CroppedImage object (1020)
829
CroppedImage object (1021)
830
CroppedImage object (1022)
831
CroppedImage object (1023)
832
CroppedImage object (1024)
833
CroppedImage object (1025)
834
CroppedImage object (1026)
835
CroppedImage object (1027)
836
CroppedImage object (1028)
837
CroppedImage object (1030)
838
CroppedImage object (1031)
839
CroppedImage object (1032)
840
CroppedImage object (1033)
841
CroppedImage object (1034)
842
CroppedImage object (1035)
843
CroppedImage object (1036)
844
CroppedImage object (1037)
845
CroppedImage object (1038)
846
CroppedImage object (1039)
847
CroppedImage object (1040)
848
CroppedImage object (1041)
849
CroppedImage object (1042)
850
CroppedImage object (1043)
851
CroppedImage object (1044)
852
CroppedImage object (1045)
853
CroppedImage object (1046)
854
CroppedImage object (1047)
855
CroppedImage object (1048)
856
CroppedImage object (1050)
857
CroppedImage object (1052)
858
CroppedImage object (1053)
859
Crop

1091
CroppedImage object (1294)
1092
CroppedImage object (1295)
1093
CroppedImage object (1296)
1094
CroppedImage object (1297)
1095
CroppedImage object (1298)
1096
CroppedImage object (1299)
1097
CroppedImage object (1300)
1098
CroppedImage object (1301)
1099
CroppedImage object (1302)
1100
CroppedImage object (1303)
1101
CroppedImage object (1304)
1102
CroppedImage object (1305)
1103
CroppedImage object (1306)
1104
CroppedImage object (1307)
1105
CroppedImage object (1308)
1106
CroppedImage object (1309)
1107
CroppedImage object (1310)
1108
CroppedImage object (1311)
1109
CroppedImage object (1312)
1110
CroppedImage object (1313)
1111
CroppedImage object (1314)
1112
CroppedImage object (1315)
1113
CroppedImage object (1316)
1114
CroppedImage object (1317)
1115
CroppedImage object (1318)
1116
CroppedImage object (1319)
1117
CroppedImage object (1320)
1118
CroppedImage object (1321)
1119
CroppedImage object (1322)
1120
CroppedImage object (1323)
1121
CroppedImage object (1324)
1122
Cro

...

4181
CroppedImage object (4608)
4182
CroppedImage object (4609)
4183
CroppedImage object (4610)
4184
CroppedImage object (4611)
4185
CroppedImage object (4612)
4186
CroppedImage object (4613)
4187
CroppedImage object (4614)
4188
CroppedImage object (4615)
4189
CroppedImage object (4616)
4190
CroppedImage object (4617)
4191
CroppedImage object (4618)
4192
CroppedImage object (4619)
4193
CroppedImage object (4620)
4194
CroppedImage object (4621)
4195
CroppedImage object (4622)
4196
CroppedImage object (4623)
4197
CroppedImage object (4624)
4198
CroppedImage object (4625)
4199
CroppedImage object (4626)
4200
CroppedImage object (4627)
4201
CroppedImage object (4628)
4202
CroppedImage object (4629)
4203
CroppedImage object (4630)
4204
CroppedImage object (4631)
4205
CroppedImage object (4632)
4206
CroppedImage object (4633)
4207
CroppedImage object (4634)
4208
CroppedImage object (4635)
4209
CroppedImage object (4636)
4210
CroppedImage object (4637)
4211
CroppedImage object (4638)
4212
Cro

4440
CroppedImage object (4883)
4441
CroppedImage object (4884)
4442
CroppedImage object (4885)
4443
CroppedImage object (4886)
4444
CroppedImage object (4887)
4445
CroppedImage object (4888)
4446
CroppedImage object (4889)
4447
CroppedImage object (4890)
4448
CroppedImage object (4891)
4449
CroppedImage object (4892)
4450
CroppedImage object (4893)
4451
CroppedImage object (4894)
4452
CroppedImage object (4895)
4453
CroppedImage object (4896)
4454
CroppedImage object (4897)
4455
CroppedImage object (4898)
4456
CroppedImage object (4899)
4457
CroppedImage object (4900)
4458
CroppedImage object (4901)
4459
CroppedImage object (4902)
4460
CroppedImage object (4903)
4461
CroppedImage object (4904)
4462
CroppedImage object (4905)
4463
CroppedImage object (4906)
4464
CroppedImage object (4907)
4465
CroppedImage object (4908)
4466
CroppedImage object (4909)
4467
CroppedImage object (4910)
4468
CroppedImage object (4911)
4469
CroppedImage object (4912)
4470
CroppedImage object (4913)
4471
Cro

4698
CroppedImage object (5147)
4699
CroppedImage object (5148)
4700
CroppedImage object (5149)
4701
CroppedImage object (5150)
4702
CroppedImage object (5151)
4703
CroppedImage object (5152)
4704
CroppedImage object (5153)
4705
CroppedImage object (5154)
4706
CroppedImage object (5155)
4707
CroppedImage object (5156)
4708
CroppedImage object (5157)
4709
CroppedImage object (5158)
4710
CroppedImage object (5159)
4711
CroppedImage object (5160)
4712
CroppedImage object (5161)
4713
CroppedImage object (5162)
4714
CroppedImage object (5163)
4715
CroppedImage object (5164)
4716
CroppedImage object (5165)
4717
CroppedImage object (5166)
4718
CroppedImage object (5167)
4719
CroppedImage object (5168)
4720
CroppedImage object (5169)
4721
CroppedImage object (5170)
4722
CroppedImage object (5171)
4723
CroppedImage object (5172)
4724
CroppedImage object (5173)
4725
CroppedImage object (5174)
4726
CroppedImage object (5175)
4727
CroppedImage object (5177)
4728
CroppedImage object (5178)
4729
Cro

4955
CroppedImage object (5409)
4956
CroppedImage object (5410)
4957
CroppedImage object (5411)
4958
CroppedImage object (5412)
4959
CroppedImage object (5413)
4960
CroppedImage object (5414)
4961
CroppedImage object (5415)
4962
CroppedImage object (5416)
4963
CroppedImage object (5417)
4964
CroppedImage object (5418)
4965
CroppedImage object (5420)
4966
CroppedImage object (5421)
4967
CroppedImage object (5422)
4968
CroppedImage object (5423)
4969
CroppedImage object (5424)
4970
CroppedImage object (5425)
4971
CroppedImage object (5426)
4972
CroppedImage object (5427)
4973
CroppedImage object (5428)
4974
CroppedImage object (5429)
4975
CroppedImage object (5430)
4976
CroppedImage object (5431)
4977
CroppedImage object (5432)
4978
CroppedImage object (5433)
4979
CroppedImage object (5434)
4980
CroppedImage object (5436)
4981
CroppedImage object (5437)
4982
CroppedImage object (5438)
4983
CroppedImage object (5439)
4984
CroppedImage object (5440)
4985
CroppedImage object (5441)
4986
Cro

5212
CroppedImage object (5678)
5213
CroppedImage object (5679)
5214
CroppedImage object (5680)
5215
CroppedImage object (5681)
5216
CroppedImage object (5682)
5217
CroppedImage object (5683)
5218
CroppedImage object (5684)
5219
CroppedImage object (5685)
5220
CroppedImage object (5686)
5221
CroppedImage object (5687)
5222
CroppedImage object (5688)
5223
CroppedImage object (5689)
5224
CroppedImage object (5690)
5225
CroppedImage object (5691)
5226
CroppedImage object (5692)
5227
CroppedImage object (5693)
5228
CroppedImage object (5694)
5229
CroppedImage object (5695)
5230
CroppedImage object (5696)
5231
CroppedImage object (5698)
5232
CroppedImage object (5699)
5233
CroppedImage object (5700)
5234
CroppedImage object (5701)
5235
CroppedImage object (5702)
5236
CroppedImage object (5703)
5237
CroppedImage object (5704)
5238
CroppedImage object (5705)
5239
CroppedImage object (5706)
5240
CroppedImage object (5707)
5241
CroppedImage object (5708)
5242
CroppedImage object (5709)
5243
Cro

In [13]:
# save_data_from_s3_temp : s3업로드 파일 박싱후 저장한 데이터 csv로 저장
# save_data_from_s3_temp('저장할 파일이름', 쿼리셋 개수) --> 쿼리셋 개수는 비워두면 전체에 대해 다운로드
# Ex: save_data_from_s3_temp("s3upload_download_test.csv","")

In [14]:
downloadmanager2= DataExtractSave()

In [15]:
downloadmanager2.save_data_from_s3_temp("s3upload_download_test.csv","")

1
CroppedImage object (5736)
2
CroppedImage object (5737)
3
CroppedImage object (5738)
4
CroppedImage object (5739)
5
CroppedImage object (5740)
6
CroppedImage object (5741)
7
CroppedImage object (5742)
8
CroppedImage object (5743)
9
CroppedImage object (5744)
10
CroppedImage object (5745)
11
CroppedImage object (5746)
12
CroppedImage object (5747)
13
CroppedImage object (5748)
14
CroppedImage object (5749)
15
CroppedImage object (5750)
16
CroppedImage object (5751)
17
CroppedImage object (5752)
18
CroppedImage object (5753)
19
CroppedImage object (5754)
20
CroppedImage object (5755)
21
CroppedImage object (5756)
22
CroppedImage object (5757)
23
CroppedImage object (5758)
24
CroppedImage object (5759)
25
CroppedImage object (5760)
26
CroppedImage object (5761)
27
CroppedImage object (5762)
28
CroppedImage object (5763)
29
CroppedImage object (5764)
30
CroppedImage object (5765)
31
CroppedImage object (5766)
32
CroppedImage object (5767)
33
CroppedImage object (5768)
34
CroppedImage obj

270
CroppedImage object (6005)
271
CroppedImage object (6006)
272
CroppedImage object (6007)
273
CroppedImage object (6008)
274
CroppedImage object (6009)
275
CroppedImage object (6010)
276
CroppedImage object (6011)
277
CroppedImage object (6012)
278
CroppedImage object (6013)
279
CroppedImage object (6014)
280
CroppedImage object (6015)
281
CroppedImage object (6016)
282
CroppedImage object (6017)
283
CroppedImage object (6018)
284
CroppedImage object (6019)
285
CroppedImage object (6020)
286
CroppedImage object (6021)
287
CroppedImage object (6022)
288
CroppedImage object (6023)
289
CroppedImage object (6024)
290
CroppedImage object (6025)
291
CroppedImage object (6026)
292
CroppedImage object (6027)
293
CroppedImage object (6028)
294
CroppedImage object (6029)
295
CroppedImage object (6030)
296
CroppedImage object (6031)
297
CroppedImage object (6032)
298
CroppedImage object (6033)
299
CroppedImage object (6034)
300
CroppedImage object (6035)
301
CroppedImage object (6036)
302
Crop

537
CroppedImage object (6273)
538
CroppedImage object (6274)
539
CroppedImage object (6275)
540
CroppedImage object (6276)
541
CroppedImage object (6277)
542
CroppedImage object (6278)
543
CroppedImage object (6279)
544
CroppedImage object (6280)
545
CroppedImage object (6281)
546
CroppedImage object (6283)
547
CroppedImage object (6284)
548
CroppedImage object (6285)
549
CroppedImage object (6286)
550
CroppedImage object (6287)
551
CroppedImage object (6288)
552
CroppedImage object (6289)
553
CroppedImage object (6290)
554
CroppedImage object (6291)
555
CroppedImage object (6292)
556
CroppedImage object (6293)
557
CroppedImage object (6294)
558
CroppedImage object (6295)
559
CroppedImage object (6296)
560
CroppedImage object (6297)
561
CroppedImage object (6298)
562
CroppedImage object (6299)
563
CroppedImage object (6300)
564
CroppedImage object (6301)
565
CroppedImage object (6302)
566
CroppedImage object (6303)
567
CroppedImage object (6304)
568
CroppedImage object (6305)
569
Crop

803
CroppedImage object (6540)
804
CroppedImage object (6541)
805
CroppedImage object (6542)
806
CroppedImage object (6543)
807
CroppedImage object (6544)
808
CroppedImage object (6545)
809
CroppedImage object (6546)
810
CroppedImage object (6547)
811
CroppedImage object (6548)
812
CroppedImage object (6549)
813
CroppedImage object (6550)
814
CroppedImage object (6551)
815
CroppedImage object (6552)
816
CroppedImage object (6553)
817
CroppedImage object (6554)
818
CroppedImage object (6555)
819
CroppedImage object (6556)
820
CroppedImage object (6557)
821
CroppedImage object (6558)
822
CroppedImage object (6559)
823
CroppedImage object (6560)
824
CroppedImage object (6561)
825
CroppedImage object (6562)
826
CroppedImage object (6563)
827
CroppedImage object (6564)
828
CroppedImage object (6565)
829
CroppedImage object (6566)
830
CroppedImage object (6567)
831
CroppedImage object (6568)
832
CroppedImage object (6569)
833
CroppedImage object (6570)
834
CroppedImage object (6571)
835
Crop

1069
CroppedImage object (6806)
1070
CroppedImage object (6807)
1071
CroppedImage object (6808)
1072
CroppedImage object (6809)
1073
CroppedImage object (6810)
1074
CroppedImage object (6811)
1075
CroppedImage object (6812)
1076
CroppedImage object (6813)
1077
CroppedImage object (6814)
1078
CroppedImage object (6815)
1079
CroppedImage object (6816)
1080
CroppedImage object (6817)
1081
CroppedImage object (6818)
1082
CroppedImage object (6819)
1083
CroppedImage object (6820)
1084
CroppedImage object (6821)
1085
CroppedImage object (6822)
1086
CroppedImage object (6823)
1087
CroppedImage object (6824)
1088
CroppedImage object (6825)
1089
CroppedImage object (6826)
1090
CroppedImage object (6827)
1091
CroppedImage object (6828)
1092
CroppedImage object (6829)
1093
CroppedImage object (6830)
1094
CroppedImage object (6831)
1095
CroppedImage object (6832)
1096
CroppedImage object (6833)
1097
CroppedImage object (6834)
1098
CroppedImage object (6835)
1099
CroppedImage object (6836)
1100
Cro

...

2100
CroppedImage object (7839)
2101
CroppedImage object (7840)
2102
CroppedImage object (7841)
2103
CroppedImage object (7842)
2104
CroppedImage object (7843)
2105
CroppedImage object (7844)
2106
CroppedImage object (7845)
2107
CroppedImage object (7846)
2108
CroppedImage object (7848)
2109
CroppedImage object (7849)
2110
CroppedImage object (7850)
2111
CroppedImage object (7851)
2112
CroppedImage object (7852)
2113
CroppedImage object (7853)
2114
CroppedImage object (7854)
2115
CroppedImage object (7855)
2116
CroppedImage object (7856)
2117
CroppedImage object (7857)
2118
CroppedImage object (7858)
2119
CroppedImage object (7859)
2120
CroppedImage object (7860)
2121
CroppedImage object (7861)
2122
CroppedImage object (7862)
2123
CroppedImage object (7863)
2124
CroppedImage object (7864)
2125
CroppedImage object (7865)
2126
CroppedImage object (7866)
2127
CroppedImage object (7867)
2128
CroppedImage object (7868)
2129
CroppedImage object (7869)
2130
CroppedImage object (7870)
2131
Cro

2357
CroppedImage object (8099)
2358
CroppedImage object (8100)
2359
CroppedImage object (8101)
2360
CroppedImage object (8102)
2361
CroppedImage object (8103)
2362
CroppedImage object (8104)
2363
CroppedImage object (8105)
2364
CroppedImage object (8106)
2365
CroppedImage object (8107)
2366
CroppedImage object (8108)
2367
CroppedImage object (8109)
2368
CroppedImage object (8110)
2369
CroppedImage object (8111)
2370
CroppedImage object (8112)
2371
CroppedImage object (8113)
2372
CroppedImage object (8114)
2373
CroppedImage object (8115)
2374
CroppedImage object (8116)
2375
CroppedImage object (8117)
2376
CroppedImage object (8118)
2377
CroppedImage object (8119)
2378
CroppedImage object (8120)
2379
CroppedImage object (8121)
2380
CroppedImage object (8122)
2381
CroppedImage object (8123)
2382
CroppedImage object (8124)
2383
CroppedImage object (8125)
2384
CroppedImage object (8126)
2385
CroppedImage object (8127)
2386
CroppedImage object (8128)
2387
CroppedImage object (8129)
2388
Cro

2616
CroppedImage object (8359)
2617
CroppedImage object (8360)
2618
CroppedImage object (8361)
2619
CroppedImage object (8362)
2620
CroppedImage object (8363)
2621
CroppedImage object (8364)
2622
CroppedImage object (8365)
2623
CroppedImage object (8366)
2624
CroppedImage object (8367)
2625
CroppedImage object (8368)
2626
CroppedImage object (8369)
2627
CroppedImage object (8370)
2628
CroppedImage object (8371)
2629
CroppedImage object (8372)
2630
CroppedImage object (8373)
2631
CroppedImage object (8374)
2632
CroppedImage object (8375)
2633
CroppedImage object (8376)
2634
CroppedImage object (8377)
2635
CroppedImage object (8378)
2636
CroppedImage object (8379)
2637
CroppedImage object (8380)
2638
CroppedImage object (8381)
2639
CroppedImage object (8382)
2640
CroppedImage object (8383)
2641
CroppedImage object (8384)
2642
CroppedImage object (8385)
2643
CroppedImage object (8386)
2644
CroppedImage object (8387)
2645
CroppedImage object (8388)
2646
CroppedImage object (8389)
2647
Cro

2873
CroppedImage object (8616)
2874
CroppedImage object (8617)
2875
CroppedImage object (8618)
2876
CroppedImage object (8619)
2877
CroppedImage object (8620)
2878
CroppedImage object (8621)
2879
CroppedImage object (8622)
2880
CroppedImage object (8623)
2881
CroppedImage object (8624)
2882
CroppedImage object (8625)
2883
CroppedImage object (8626)
2884
CroppedImage object (8627)
2885
CroppedImage object (8628)
2886
CroppedImage object (8629)
2887
CroppedImage object (8630)
2888
CroppedImage object (8631)
2889
CroppedImage object (8632)
2890
CroppedImage object (8633)
2891
CroppedImage object (8634)
2892
CroppedImage object (8635)
2893
CroppedImage object (8636)
2894
CroppedImage object (8637)
2895
CroppedImage object (8638)
2896
CroppedImage object (8639)
2897
CroppedImage object (8640)
2898
CroppedImage object (8641)
2899
CroppedImage object (8642)
2900
CroppedImage object (8643)
2901
CroppedImage object (8644)
2902
CroppedImage object (8645)
2903
CroppedImage object (8646)
2904
Cro

3130
CroppedImage object (8873)
3131
CroppedImage object (8874)
3132
CroppedImage object (8875)
3133
CroppedImage object (8876)
3134
CroppedImage object (8877)
3135
CroppedImage object (8878)
3136
CroppedImage object (8879)
3137
CroppedImage object (8880)
3138
CroppedImage object (8881)
3139
CroppedImage object (8882)
3140
CroppedImage object (8883)
3141
CroppedImage object (8884)
3142
CroppedImage object (8885)
3143
CroppedImage object (8886)
3144
CroppedImage object (8887)
3145
CroppedImage object (8888)
3146
CroppedImage object (8889)
3147
CroppedImage object (8890)
3148
CroppedImage object (8891)
3149
CroppedImage object (8892)
3150
CroppedImage object (8893)
3151
CroppedImage object (8894)
3152
CroppedImage object (8895)
3153
CroppedImage object (8896)
3154
CroppedImage object (8897)
3155
CroppedImage object (8898)
3156
CroppedImage object (8899)
3157
CroppedImage object (8900)
3158
CroppedImage object (8901)
3159
CroppedImage object (8902)
3160
CroppedImage object (8903)
3161
Cro

In [20]:
CroppedImage.objects.filter(origin_source__image="").exclude(origin_source__image="").count()

0

In [6]:
CroppedImage.objects.filter(origin_source__image_url__isnull=False).count()

8565

In [10]:
CroppedImage.objects.all().count()

8565

In [18]:
pd.read_csv('s3upload_download_test.csv').count()

left          3271
top           3271
right         3271
bottom        3271
origin_url    3271
width         3271
height        3271
dtype: int64